# Decode 'langauge' feature

In [10]:
import sys
sys.path.append('..')

from utils.cuda_cluster import client
from utils.util import plot_graphs
import core.config as conf
from utils.dataset import read_data, factorize_small_cardinality

import numpy as np
from tqdm import tqdm
from datetime import datetime 
import matplotlib.pyplot as plt

import tensorflow as tf
from transformers import *
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint


In [8]:
client

Client Scheduler: tcp://127.0.0.1:39629 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 1 Cores: 1 Memory: 33.47 GB


In [90]:
#random seed 
tf.random.set_seed(1234)
np.random.seed(1234)

In [91]:
tokenizer = BertTokenizer.from_pretrained("bert-base-multilingual-cased", cache_dir='bert_ckpt', do_lower_case=False)

Downloading: 100%|██████████| 996k/996k [00:01<00:00, 845kB/s] 
Downloading: 100%|██████████| 29.0/29.0 [00:00<00:00, 40.2kB/s]
Downloading: 100%|██████████| 1.96M/1.96M [00:01<00:00, 1.52MB/s]


## 1. Load data

In [249]:
# data_path = conf.raw_data_path + '*' # for all dataset
data_path = conf.raw_data_path + 'part-00197'
ori_df = read_data(data_path)

number of rows: 3034025


In [250]:
# ori_df = ori_df[ori_df['tweet_type'] == 'TopLevel']
df = ori_df[['text_ tokens', 'language']]

In [251]:
df.head()

,text_ tokens,language
id,,
1,101\t12489\t112\t187\t20442\t55768\t85377\t101...,488B32D24BD4BB44172EB981C1BCA6FA
2,101\t56898\t137\t32916\t11359\t11359\t131\t199...,E7F038DE3EAD397AEC9193686C911677
3,101\t1413\t18427\t111434\t46856\t20503\t58264\...,9FCF19233EAD65EA6E32C2E6DC03A444
4,101\t56658\t11127\t18137\t66014\t30340\t12396\...,488B32D24BD4BB44172EB981C1BCA6FA
5,101\t14120\t131\t120\t120\t188\t119\t11170\t12...,313ECD3A1E5BB07406E4249475C2D6D6


In [254]:
df['cnt'] = 1

## Get unique language

In [270]:
languages_cnt = df.compute().groupby('language')[['cnt']].count().reset_index()
languages_tokens = df.compute().groupby('language')[['text_ tokens']].nth(-1).reset_index()
languages = languages_tokens.merge(languages_cnt, on=['language'], how='left')


In [273]:
languages = languages.sort_values('cnt', ascending=False)
languages['id'] = range(66)
languages = languages.set_index('id')

In [274]:
languages

,language,text_ tokens,cnt
id,,,
0,488B32D24BD4BB44172EB981C1BCA6FA,101\t18740\t11003\t98514\t10529\t12229\t18103\...,1096401
1,E7F038DE3EAD397AEC9193686C911677,101\t1972\t18825\t3425\t100\t14120\t131\t120\t...,520269
2,B0FA488F2911701DD8EC5B1EA5E322D8,101\t77603\t10165\t85912\t10125\t11639\t75366\...,253290
3,B8B04128918BBF54E2E178BFF1ABA833,101\t100\t108\t16062\t11281\t10115\t11274\t102...,242549
4,313ECD3A1E5BB07406E4249475C2D6D6,101\t77603\t10133\t136\t14120\t131\t120\t120\t...,194984
...,...,...,...
61,CDE47D81F953D800F760F1DE8AA754BA,101\t100\t14120\t131\t120\t120\t188\t119\t1117...,28
62,9D831A0F3603A54732CCBDBF291D17B7,101\t100\t14120\t131\t120\t120\t188\t119\t1117...,20
63,5F152815982885A996841493F2757D91,101\t100\t216\t137\t86545\t11090\t168\t12464\t102,7


In [275]:
tokens = list(map(lambda x: x.split('\t'), languages['text_ tokens'].to_array()))
decoded_tokens = []
n_languages = len(languages)
languages['language_id'] = range(n_languages)
for i in range(n_languages):
    decoded_tokens.append(tokenizer.decode(tokens[i], skip_special_tokens=True))

In [276]:
languages['decoded_tweet_tokens'] = decoded_tokens

In [317]:
df[df['language'] == '313ECD3A1E5BB07406E4249475C2D6D6'].compute()

a = list(map(int, df.loc[3033944]['text_ tokens'].compute().to_array()[0].split('\t')))
print(tokenizer.decode(a))

[CLS] Rašid Hadžović, inače ministar OBRAZOVANJA u Hercegovačko neretvanskoj županiji. https : / / t. co / GcvEpKQ3t0 [SEP]


In [318]:
language_type = [
    '영어', # 0
    '일본어', # 1
    '스페인어', # 2
    '포루투갈어', # 3
    '크로아티아어', # 4
    '터키어', # 5
    '태국어', # 6
    '아랍어', # 7
    '한국어', # 8
    '프랑스어', # 9
    '인도네시아어', # 10
    '힌디어', # 11
    '러시아어', # 12
    '타갈로그어', # 13
    '이탈리아어', # 14
    '페르시아어', # 15
    '독일어', # 16
    '폴란드어', # 17
    '우르두어', # 18
    '네덜란드어', # 19
    '카탈로니아어', # 20
    '중국어', # 21
    '타밀어', # 22
    '?', # 23
    '?', # 24
    '그리스어', # 25
    '터키어', # 26
    '체코어', # 27
    '히브리어', # 28
    '핀란드어', # 29
    '노르웨이어', # 30
    '마라티어', # 31
    '텔루구어', # 32
    '?', # 33
    '우크라이나어', # 34
    '?', # 35
    '네팔어', # 36
    '노르웨이어', # 37
    '?', # 38
    '헝가리어', # 39
    '베트남어', # 40
    '버마어', # 41
    '프랑스어', # 42
    '포르투갈어', # 43
    '마케도니아어', # 44
    '?', # 45
    '말라가시어', # 46
    '구자라트어', # 47
    '말라얄람어', # 48
    '칸나다어', # 49
    '스와힐리어', # 50
    '펀잡어', # 51
    '불가리아어', # 52
    '페르시아어', # 53
    '페르시아어', # 54
    '신디어', # 55
    '?', # 56
    '?', # 57
    '?', # 58
    '?', # 59
    '아르메니아어', # 60
    '?', # 61
    '?', # 62
    '?', # 63
    '위구르어', # 64
    '?', # 65
]

In [319]:
languages['language_type'] = language_type

In [320]:
res = languages[['language_id', 'language', 'language_type']]

In [321]:
res.to_csv(conf.project_root + 'data/language.csv', index=False)